In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

We performed an unsupervised KMeans clustering on stacked ecological raster data for the Amazonian basin, using bioclimatic variables, Brazil nut habitat suitability, and tree species richness. Five distinct environmental clusters were identified, which we then overlaid with known archaeological site locations derived from field surveys. By analyzing the spatial distribution and environmental profiles of each cluster, we identified Clusters 0 and 3 as being most strongly associated with archaeological presence. These clusters exhibit high tree species richness and notable Brazil nut suitability, suggesting ancient human influence such as agroforestry or terra preta formation. We used GPT-4 to interpret the clustering results, confirm anthropogenic relevance, and recommend priority zones for predictive modeling of undiscovered archaeological sites.

In [ ]:
!pip install rasterio

In [ ]:
!pip install localtileserver

In [ ]:
import rasterio
import numpy as np
import os
from rasterio.merge import merge
import matplotlib.pyplot as plt

In [ ]:
import rasterio
import numpy as np
import os
from glob import glob

# Step 1: Define file paths (sorted for consistency)
bioclim_paths = sorted(glob("/kaggle/input/bioclims/bio*.tif"))

# Step 2: Read and stack rasters
arrays = []
meta = None

for path in bioclim_paths:
    with rasterio.open(path) as src:
        if meta is None:
            meta = src.meta.copy()
        arrays.append(src.read(1))  # read first band

# Step 3: Convert to numpy stack (shape: bands x height x width)
bioclim_stack = np.stack(arrays, axis=0)

# Update metadata to reflect stacked bands
meta.update(count=len(bioclim_paths))

print("✅ Loaded stack shape:", bioclim_stack.shape)
print("Bands:", [os.path.basename(p) for p in bioclim_paths])


In [ ]:
import matplotlib.pyplot as plt

# Step 1: Use band_dict and band_names from earlier
# (redefine if necessary)
band_names = [os.path.basename(p).replace(".tif", "") for p in bioclim_paths]
band_dict = dict(zip(band_names, bioclim_stack))

# Step 2: Plot all bands
n = len(band_names)
cols = 4
rows = int(np.ceil(n / cols))

plt.figure(figsize=(4 * cols, 4 * rows))
for i, name in enumerate(band_names):
    plt.subplot(rows, cols, i + 1)
    plt.imshow(band_dict[name], cmap='viridis')
    plt.title(name)
    plt.axis('off')

plt.suptitle("Bioclimatic Variables", fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
treerich_path='/kaggle/input/amazontrees/treerich1.tif'

In [ ]:
import rasterio

with rasterio.open(treerich_path) as src:
    treerich_data = src.read(1)  # Read the first band
    treerich_meta = src.meta     # Get metadata (crs, transform, dtype, etc.)

print("✅ Shape:", treerich_data.shape)
print("✅ CRS:", treerich_meta['crs'])


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
plt.imshow(treerich_data, cmap='YlGn', vmin=0)
plt.title("🌳 Tree Species Richness")
plt.colorbar(label='Species Richness')
plt.axis('off')
plt.show()


In [ ]:
brzl_path='/kaggle/input/brazilnut-sdm/mergedBrazilNut_sdm.tif'

In [ ]:
import rasterio

with rasterio.open(brzl_path) as src:
    brzl_data = src.read(1)  # Read the first band
    brzl_meta = src.meta     # Get metadata (crs, transform, dtype, etc.)

print("✅ Shape:", brzl_data.shape)
print("✅ CRS:", brzl_meta['crs'])

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
plt.imshow(brzl_data, cmap='YlGn', vmin=0)
plt.title("Brazil Tree")
plt.colorbar(label='HSM')
plt.axis('off')
plt.show()


In [ ]:
import numpy as np
import rasterio
from rasterio.warp import reproject, Resampling
import matplotlib.pyplot as plt

# Paths
bio_paths = [
    '/kaggle/input/bioclims/bio02.tif',
    '/kaggle/input/bioclims/bio03.tif',
    '/kaggle/input/bioclims/bio04.tif',
    '/kaggle/input/bioclims/bio05.tif',
    '/kaggle/input/bioclims/bio07.tif',
    '/kaggle/input/bioclims/bio15.tif',
    '/kaggle/input/bioclims/bio18.tif',
    '/kaggle/input/bioclims/bio19.tif'
]
suitability_path = '/kaggle/input/brazilnut-sdm/mergedBrazilNut_sdm.tif'
treerich_path = '/kaggle/input/amazontrees/treerich1.tif'

# Step 1: Load bioclim stack
bio_stack = []
for path in bio_paths:
    with rasterio.open(path) as src:
        bio_stack.append(src.read(1))
        ref_meta = src.meta

bio_stack = np.stack(bio_stack)

# Step 2: Function to resample raster to match reference
def resample_to_match(src_path, ref_meta):
    with rasterio.open(src_path) as src:
        raw = src.read(1)
        dst = np.empty((ref_meta['height'], ref_meta['width']), dtype=np.float32)

        reproject(
            source=raw,
            destination=dst,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=ref_meta['transform'],
            dst_crs=ref_meta['crs'],
            resampling=Resampling.bilinear
        )
    return dst

# Step 3: Resample suitability and tree richness
brzl_resampled = resample_to_match(suitability_path, ref_meta)
treerich_resampled = resample_to_match(treerich_path, ref_meta)

# Step 4: Stack all layers
combined_stack = np.concatenate([
    bio_stack,
    brzl_resampled[np.newaxis, ...],
    treerich_resampled[np.newaxis, ...]
], axis=0)

# Step 5: Visualize key layers
fig, axes = plt.subplots(1, 4, figsize=(20, 5))
titles = ['bio19', 'bio18', 'Suitability', 'Tree Richness']
band_idx = [7, 6, 8, 9]  # Indices in stack

for i, ax in enumerate(axes):
    im = ax.imshow(combined_stack[band_idx[i]], cmap='viridis')
    ax.set_title(titles[i])
    ax.axis("off")
    plt.colorbar(im, ax=ax, shrink=0.7)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Step 1: Get shape
bands, height, width = combined_stack.shape

# Step 2: Flatten the stack: (n_pixels, n_features)
flat_data = combined_stack.reshape(bands, -1).T  # Shape: (height * width, bands)

# Step 3: Create mask for valid pixels
valid_mask = ~np.any(np.isnan(flat_data), axis=1) & (np.any(flat_data != 0, axis=1))

# Step 4: Apply KMeans on valid pixels
X = flat_data[valid_mask]
kmeans = KMeans(n_clusters=5, random_state=42, n_init='auto')
labels = np.full(flat_data.shape[0], fill_value=-1, dtype=np.int32)  # Use -1 for nodata
labels[valid_mask] = kmeans.fit_predict(X)

# Step 5: Reshape labels to 2D map
cluster_map = labels.reshape(height, width)

# Step 6: Visualize result
plt.figure(figsize=(10, 7))
plt.imshow(cluster_map, cmap='tab10')
plt.title("KMeans Clusters (k=5)")
plt.axis("off")
plt.colorbar(label="Cluster ID")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd

# Step 1: Get number of features (bands)
num_features = combined_stack.shape[0]

# Step 2: Reshape for per-pixel access
# Shape: (n_pixels, n_features)
flat_features = combined_stack.reshape(num_features, -1).T  # (n_pixels, n_features)

# Step 3: Reuse cluster labels
valid_features = flat_features[valid_mask]
valid_labels = labels[valid_mask]

# Step 4: Compute mean feature values per cluster
cluster_profiles = []

for cluster_id in np.unique(valid_labels):
    cluster_pixels = valid_features[valid_labels == cluster_id]
    mean_profile = np.nanmean(cluster_pixels, axis=0)
    cluster_profiles.append((cluster_id, mean_profile))

# Step 5: Display as table
df_profiles = pd.DataFrame(
    [profile for _, profile in cluster_profiles],
    index=[f"Cluster {cid}" for cid, _ in cluster_profiles],
    columns=[f"Feature {i}" for i in range(num_features)]
)

# Optional: round and display
print(df_profiles.round(2))


In [ ]:
df_profiles.columns = [
    "BIO02 (Diurnal Range)",
    "BIO03 (Isothermality)",
    "BIO04 (Temp Seasonality)",
    "BIO05 (Max Temp Warmest Month)",
    "BIO07 (Temp Annual Range)",
    "BIO15 (Precip Seasonality)",
    "BIO18 (Precip Warmest Quarter)",
    "BIO19 (Precip Coldest Quarter)",
    "Brazil Nut Suitability",
    "Tree Species Richness"
]


In [ ]:
markdown_text = df_profiles.round(2).to_markdown()
print(markdown_text)


In [ ]:
!pip install openai --upgrade


In [ ]:
import openai

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("OpenAI")

In [ ]:
import os
from openai import OpenAI
from kaggle_secrets import UserSecretsClient

# Access OpenAI key from Kaggle Secrets
user_secrets = UserSecretsClient()
openai_key = user_secrets.get_secret("OpenAI")  # "OpenAI" should be the secret label

# Create OpenAI client
client = OpenAI(api_key=openai_key)

In [ ]:
prompt = """
You're a geospatial archaeologist analyzing 5 landscape clusters in Amazon basin. Each cluster represents a unique combination of environmental and ecological conditions, derived from bioclimatic variables, Brazil nut habitat suitability, and tree species richness.

Below are the definitions of the features used to characterize each cluster:

- Feature 0: BIO02 – Mean Diurnal Temperature Range
- Feature 1: BIO03 – Isothermality
- Feature 2: BIO04 – Temperature Seasonality
- Feature 3: BIO05 – Max Temperature of Warmest Month
- Feature 4: BIO07 – Temperature Annual Range
- Feature 5: BIO15 – Precipitation Seasonality
- Feature 6: BIO18 – Precipitation of Warmest Quarter
- Feature 7: BIO19 – Precipitation of Coldest Quarter
- Feature 8: Brazil Nut Habitat Suitability (0–1)
- Feature 9: Tree Species Richness

Your task is to analyze the mean profile of each cluster (below) and answer the following:

1. Which clusters are ecologically unusual or anomalous?
2. Which clusters might correspond to anthropogenic or archaeologically relevant zones (e.g., suitable for agroforestry, terra preta, or geoglyph contexts)?
3. Which clusters likely represent natural vs. human-managed forests, and why?

Here are the cluster profiles:

Cluster 0:
  BIO02: 11.79, BIO03: 72.68, BIO04: 11.22, BIO05: 31.55, BIO07: 16.42
  BIO15: 62.26, BIO18: 450.34, BIO19: 180.90
  Brazil Nut Suitability: 0.06, Tree Richness: 47.81

Cluster 1:
  BIO02: 11.88, BIO03: 74.95, BIO04: 9.06, BIO05: 29.65, BIO07: 15.98
  BIO15: 67.15, BIO18: 366.93, BIO19: 331.35
  Brazil Nut Suitability: 0.06, Tree Richness: -9950.94

Cluster 2:
  BIO02: 10.57, BIO03: 78.11, BIO04: 6.28, BIO05: 32.79, BIO07: 13.68
  BIO15: 53.18, BIO18: 400.29, BIO19: 576.44
  Brazil Nut Suitability: 0.11, Tree Richness: -2973.82

Cluster 3:
  BIO02: 9.71, BIO03: 81.24, BIO04: 4.85, BIO05: 32.50, BIO07: 11.95
  BIO15: 45.83, BIO18: 380.33, BIO19: 882.61
  Brazil Nut Suitability: 0.06, Tree Richness: 98.90

Cluster 4:
  BIO02: 10.65, BIO03: 78.01, BIO04: 6.55, BIO05: 32.62, BIO07: 13.80
  BIO15: 54.54, BIO18: 397.00, BIO19: 568.19
  Brazil Nut Suitability: 0.12, Tree Richness: -6310.92
"""


In [ ]:
!pip install --upgrade openai


In [ ]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a geospatial archaeologist."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.4
)

print(response.choices[0].message.content)


Re-run after data cleaning

In [ ]:
df_profiles["Tree Species Richness"] = df_profiles["Tree Species Richness"].clip(lower=0)


In [ ]:
df_profiles.head()

In [ ]:
cluster_labels = {
    0: "Possible managed agroforestry zone",
    1: "Natural forest (low richness)",
    2: "Natural forest (low richness)",
    3: "Possible human-modified rich zone",
    4: "Natural forest (low richness)"
}


In [ ]:
from openai import OpenAI
from kaggle_secrets import UserSecretsClient

# Access API key securely
user_secrets = UserSecretsClient()
openai_key = user_secrets.get_secret("OpenAI")

client = OpenAI(api_key=openai_key)

# Final GPT prompt with correct values
prompt = """
You're a geospatial archaeologist analyzing 5 landscape clusters in Acre, Brazil. Each cluster is derived from bioclimatic variables, Brazil nut suitability, and tree species richness, and may reflect natural or human-influenced landscape patterns.

Feature definitions:
- BIO02: Mean Diurnal Temperature Range
- BIO03: Isothermality
- BIO04: Temperature Seasonality
- BIO05: Max Temperature of Warmest Month
- BIO07: Temperature Annual Range
- BIO15: Precipitation Seasonality
- BIO18: Precipitation of Warmest Quarter
- BIO19: Precipitation of Coldest Quarter
- Brazil Nut Suitability: Ranges from 0 (low) to 1 (high suitability)
- Tree Species Richness: Number of unique tree species

Please analyze the clusters and answer:
1. Which clusters are ecologically unusual or anomalous?
2. Which clusters might correspond to archaeologically relevant zones (e.g., agroforestry, geoglyphs, terra preta)?
3. Which clusters likely represent natural vs. human-managed forests, and why?

Cluster 0:
BIO02: 11.79, BIO03: 72.68, BIO04: 11.22, BIO05: 31.55, BIO07: 16.42
BIO15: 62.26, BIO18: 450.34, BIO19: 180.90
Brazil Nut Suitability: 0.0558, Tree Species Richness: 47.81

Cluster 1:
BIO02: 11.88, BIO03: 74.95, BIO04: 9.06, BIO05: 29.65, BIO07: 15.98
BIO15: 67.15, BIO18: 366.93, BIO19: 331.35
Brazil Nut Suitability: 0.0555, Tree Species Richness: 0.00

Cluster 2:
BIO02: 10.57, BIO03: 78.11, BIO04: 6.28, BIO05: 32.79, BIO07: 13.68
BIO15: 53.18, BIO18: 400.29, BIO19: 576.44
Brazil Nut Suitability: 0.1131, Tree Species Richness: 0.00

Cluster 3:
BIO02: 9.71, BIO03: 81.24, BIO04: 4.85, BIO05: 32.50, BIO07: 11.95
BIO15: 45.83, BIO18: 380.33, BIO19: 882.61
Brazil Nut Suitability: 0.0554, Tree Species Richness: 98.90

Cluster 4:
BIO02: 10.65, BIO03: 78.01, BIO04: 6.55, BIO05: 32.62, BIO07: 13.80
BIO15: 54.54, BIO18: 397.00, BIO19: 568.19
Brazil Nut Suitability: 0.1245, Tree Species Richness: 0.00
"""

# Query GPT-4
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a geospatial archaeologist."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.3
)

# Print GPT's interpretation
print(response.choices[0].message.content)


**Archeological types**

In [ ]:
import pandas as pd
gdf = pd.read_csv('/kaggle/input/archeologicaltype/filtered_points.csv')


In [ ]:
gdf.head()

In [ ]:
# Create GeoDataFrame
gdf['geometry'] = gdf.apply(lambda row: Point(row['POINT_X'], row['POINT_Y']), axis=1)
gdf = gpd.GeoDataFrame(gdf, geometry='geometry', crs='EPSG:4326')

In [ ]:
import rasterio

with rasterio.open('/kaggle/input/bioclims/bio02.tif') as src:
    transform = src.transform
    raster_crs = src.crs



In [ ]:
# Reproject site GeoDataFrame to match raster CRS
gdf = gdf.to_crs(raster_crs)

# Convert point coords to raster pixel indices
def coords_to_rc(x, y, transform):
    col, row = ~transform * (x, y)
    return int(row), int(col)

# Generate (row, col) for each site
rows_cols = [coords_to_rc(x, y, transform) for x, y in zip(gdf.geometry.x, gdf.geometry.y)]

# Extract cluster ID from in-memory cluster_map
gdf['cluster_id'] = [
    cluster_map[r, c] if 0 <= r < cluster_map.shape[0] and 0 <= c < cluster_map.shape[1] else -1
    for r, c in rows_cols
]


In [ ]:
import matplotlib.pyplot as plt
from rasterio.transform import rowcol

# 1. Make sure gdf is in the same CRS as the raster
gdf_proj = gdf.to_crs(raster_crs)

# 2. Convert lat/lon → raster pixel (row, col)
def lonlat_to_pixel(x, y, transform):
    col, row = ~transform * (x, y)
    return int(col), int(row)

pixel_coords = [lonlat_to_pixel(x, y, transform) for x, y in zip(gdf_proj.geometry.x, gdf_proj.geometry.y)]
cols, rows = zip(*pixel_coords)  # flip for plotting

# 3. Plot raster with scatter
plt.figure(figsize=(10, 8))
plt.imshow(cluster_map, cmap='tab10')
plt.scatter(cols, rows, c='black', s=10, label='Field Sites (pixels)')
plt.title('Cluster Map with Correctly Positioned Field Sites')
plt.legend()
plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
plt.savefig("cluster_sites_overlay.png", dpi=300, bbox_inches='tight')


In [ ]:
import os
os.listdir("/kaggle/working/")


In [ ]:
gdf['cluster_id'].value_counts().sort_index()


In [ ]:
df_summary = df_profiles.copy()
df_summary["Site Count"] = gdf["cluster_id"].value_counts().reindex(df_summary.index).fillna(0).astype(int)
df_summary = df_summary.round(2)



In [ ]:
print(df_summary.to_markdown(tablefmt="github"))


In [ ]:
import pandas as pd
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
import os

# Save the image (if not already saved)
plt.savefig("cluster_sites_overlay.png", dpi=300, bbox_inches="tight")

# Combine df_profiles and gdf['cluster_id'] count
df_summary = df_profiles.copy()
df_summary["Site Count"] = gdf["cluster_id"].value_counts().reindex(df_profiles.index).fillna(0).astype(int)
df_summary = df_summary.round(2)

# Optional: rename index
df_summary.index = [f"Cluster {i}" for i in df_summary.index]

# Convert to markdown table string
table_md = df_summary.reset_index().to_markdown(index=False)

# Create full markdown block
markdown_text = f"""
## 📍 Cluster Map with Archaeological Site Overlays

This map shows the 5 KMeans-derived environmental clusters across Acre, Brazil.  
Each **black dot** represents a known archaeological site (e.g., platform mounds), overlaid using georeferenced field data.

![Cluster Overlay](cluster_sites_overlay.png)

---

## 📊 Cluster Summary Table

{table_md}

---

## 🤖 GPT Prompt for Interpretation

> Based on the table and spatial pattern of sites, which clusters are most strongly associated with known archaeological presence?  
> Which ones suggest evidence of ancient human land use (e.g., agroforestry, terra preta)?  
> Which are likely natural background zones?  
> Recommend priority clusters for predictive modeling of new archaeological sites.
"""

# Display in notebook
display(Markdown(markdown_text))


In [ ]:
from openai import OpenAI

# Authenticate your OpenAI client
client = OpenAI(api_key=openai_key)  # assumes you've loaded your key already

# Send the markdown text (generated above) to GPT-4
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a geospatial archaeologist."},
        {"role": "user", "content": markdown_text}  # send your markdown block directly
    ],
    temperature=0.3,
    max_tokens=1500  # optional, increase if response cuts off
)

# Output GPT's interpretation
print(response.choices[0].message.content)


In [ ]:
prompt = f"""
You are a geospatial archaeologist analyzing five environmental landscape clusters in Acre, Brazil. Each cluster was derived from KMeans clustering of ecological raster data, including bioclimatic variables, Brazil nut habitat suitability, and tree species richness.

The figure attached below — *cluster_sites_overlay.png* — shows the spatial distribution of these clusters. **Black points** represent known archaeological sites (e.g., platform mounds) based on field survey data.

---

### Known Archaeological Site Counts:
- Cluster 0 → **450 sites**
- Cluster 1 → **62 sites**
- Cluster 2 → **61 sites**
- Cluster 3 → **293 sites**
- Cluster 4 → **54 sites**

---

### Please Analyze:

1. Which clusters show the strongest known archaeological presence?
2. Which clusters are likely influenced by ancient human activity (e.g., agroforestry, terra preta)?
3. Which clusters reflect natural or background ecological zones?
4. Based on this, which clusters should be prioritized for predictive modeling of new archaeological sites?

Note: High Brazil nut suitability and tree species richness are potential proxies for ancient agroforestry.

---

### Environmental Profiles by Cluster:

{table_md}
"""


In [ ]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a geospatial archaeologist."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.3,
    max_tokens=1500
)

print(response.choices[0].message.content)
